<a href="https://colab.research.google.com/github/luismiguelmartinluengo/PySpark_Demos/blob/main/Labo_Sql_eCommerce_Organization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import lower, upper, avg, min, max, sum

In [2]:
sparkSession = SparkSession.builder.appName('Labo Sql eCommerce Organization').getOrCreate()

In [37]:
customers = sparkSession.read.csv('/content/drive/MyDrive/Colab Notebooks/data/customers.csv',
                                  header = True,
                                  inferSchema = True)
customers.printSchema()
customers.show(5)

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- join_date: date (nullable = true)
 |-- country: string (nullable = true)
 |-- loyalty_points: integer (nullable = true)

+-----------+-----------------+--------------------+----------+-------------+--------------+
|customer_id|             name|               email| join_date|      country|loyalty_points|
+-----------+-----------------+--------------------+----------+-------------+--------------+
|          1|     Norma Fisher| ysullivan@yahoo.com|2023-10-23|Guinea-Bissau|          6311|
|          2|     Susan Wagner|katelynmontgomery...|2024-04-14|      Romania|          6890|
|          3|Stephanie Collins|thomas15@stewart-...|2024-06-15|      Lesotho|           663|
|          4|     Joseph Brown|cortezraymond@gar...|2021-10-30|  Saint Lucia|          4242|
|          5|         Seth Lee|lindathomas@west.net|2023-12-02|     Botswana|          8376|
+-------

In [38]:
purchases = sparkSession.read.csv('/content/drive/MyDrive/Colab Notebooks/data/purchases.csv',
                                  header = True,
                                  inferSchema = True)
purchases.printSchema()
purchases.show(5)

root
 |-- purchase_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- purchase_amount: double (nullable = true)
 |-- purchase_date: date (nullable = true)
 |-- product_category: string (nullable = true)
 |-- payment_method: string (nullable = true)

+-----------+-----------+---------------+-------------+----------------+--------------+
|purchase_id|customer_id|purchase_amount|purchase_date|product_category|payment_method|
+-----------+-----------+---------------+-------------+----------------+--------------+
|          1|         67|          263.9|   2023-09-12|          Beauty|        PayPal|
|          2|        240|         192.87|   2023-11-05|           Books|        PayPal|
|          3|        597|          93.98|   2024-09-28|     Electronics| Bank Transfer|
|          4|        517|         170.66|   2024-04-25|            Toys|   Credit Card|
|          5|        832|          73.61|   2023-02-20|           Books|    Debit Card|
+-----------+-----

In [39]:
#Esta notación no unifica el campo de unión y obliga a quitarlo explícitamente para tener un dataframe limpio. No me gusta
#df = customers.join(purchases, customers.customer_id == purchases.customer_id, how='inner').drop(purchases.customer_id)
#Mejor usar la que es similar a pandas.merge
cp = customers.join(purchases, on = 'customer_id', how='inner')
cp.show(5)

+-----------+-----------------+--------------------+----------+-------------+--------------+-----------+---------------+-------------+----------------+--------------+
|customer_id|             name|               email| join_date|      country|loyalty_points|purchase_id|purchase_amount|purchase_date|product_category|payment_method|
+-----------+-----------------+--------------------+----------+-------------+--------------+-----------+---------------+-------------+----------------+--------------+
|          1|     Norma Fisher| ysullivan@yahoo.com|2023-10-23|Guinea-Bissau|          6311|        642|         317.67|   2023-02-15|          Beauty|        PayPal|
|          2|     Susan Wagner|katelynmontgomery...|2024-04-14|      Romania|          6890|        455|         407.47|   2023-08-20|         Fashion|     Gift Card|
|          3|Stephanie Collins|thomas15@stewart-...|2024-06-15|      Lesotho|           663|        714|          157.4|   2024-03-02|          Beauty|    Debit Card

In [40]:
cp.createOrReplaceTempView('cp')
customers.createOrReplaceTempView('customers')
purchases.createOrReplaceTempView('purchases')

In [41]:
sqlCustomersTopPurchase = '''SELECT name AS Name, SUM(purchase_amount) as TotalPurchase
                            FROM cp
                            WHERE PURCHASE_DATE >= "2024-01-01"
                            GROUP BY name
                            ORDER BY TotalPurchase DESC
                            LIMIT 3'''
sparkSession.sql(sqlCustomersTopPurchase).show()

+---------------+-------------+
|           Name|TotalPurchase|
+---------------+-------------+
|      Mark Hart|      1301.17|
|     Kevin Wade|        928.6|
|Jonathan Gaines|       926.78|
+---------------+-------------+



In [42]:
sqlCustomersTopAvgPurchase = '''SELECT name AS Name, AVG(purchase_amount) as AveragePurchase
                            FROM cp
                            WHERE PURCHASE_DATE >= "2024-01-01"
                            GROUP BY name
                            ORDER BY AveragePurchase DESC
                            LIMIT 3'''
sparkSession.sql(sqlCustomersTopAvgPurchase).show()

+--------------+---------------+
|          Name|AveragePurchase|
+--------------+---------------+
|  Diane Palmer|         498.16|
|Jessica Dorsey|         493.67|
| Sharon Farmer|         488.77|
+--------------+---------------+



In [55]:
#Madre mía, tienen la consulta mal. Travis Schultz no es cliente inactivo y sale en los resultados.
sqlInactiveCustomers = '''SELECT c.customer_id, c.name, c.email
                          FROM Customers c
                            LEFT JOIN Purchases p ON c.customer_id = p.customer_id
                          WHERE p.purchase_date IS NULL
                             OR p.purchase_date < "2023-01-01"'''
sparkSession.sql(sqlInactiveCustomers).show(10)
#demostración del error
purchases.filter(purchases.customer_id == 18).show(100, truncate = False)

+-----------+------------------+--------------------+
|customer_id|              name|               email|
+-----------+------------------+--------------------+
|         11|    Susan Ferguson|martinezjacob@wil...|
|         15|     Michael Bruce|   david72@gmail.com|
|         16|      Keith Romero|  paul42@hotmail.com|
|         17|Gabriella Williams|johnbenton@yahoo.com|
|         18|    Travis Schultz|  paul61@wheeler.com|
|         21|     Sheri Burnett|monroedavid@moral...|
|         24|       Diana Smith|eringrant@mays-br...|
|         26|   Richard Johnson|edwardsilva@yahoo...|
|         27|       Brian Cross|hayden81@powell-m...|
|         28|      Karen Rogers|mendozaholly@yaho...|
+-----------+------------------+--------------------+
only showing top 10 rows

+-----------+-----------+---------------+-------------+----------------+--------------+
|purchase_id|customer_id|purchase_amount|purchase_date|product_category|payment_method|
+-----------+-----------+---------------+-

In [65]:
#consulta de clientes inactivos alternativa
sqlInactiveCustomersAlt = '''SELECT c.customer_id, c.name, c.email, MAX(p.purchase_date) as LastPurchaseDate
                          FROM Customers c
                            LEFT JOIN Purchases p ON c.customer_id = p.customer_id
                          GROUP BY c.customer_id, c.name, c.email
                          HAVING (LastPurchaseDate IS NULL OR LastPurchaseDate < "2023-01-01")'''
sparkSession.sql(sqlInactiveCustomersAlt).show(10)

+-----------+--------------------+--------------------+----------------+
|customer_id|                name|               email|LastPurchaseDate|
+-----------+--------------------+--------------------+----------------+
|        115|    Margaret Sanchez|webbpamela@hotmai...|            NULL|
|        389|           John Knox|johnwilliams@brow...|            NULL|
|        956|       Andrew Pierce|ramirezaustin@yah...|            NULL|
|        586|Mr. Michael Mccul...| dbryan@martinez.net|            NULL|
|        718|          Kathryn Li|elizabethphillips...|            NULL|
|        248|   Kristopher Castro|ronalddrake@hotma...|            NULL|
|        589|    Jacqueline Walls|robertsims@sulliv...|            NULL|
|         11|      Susan Ferguson|martinezjacob@wil...|            NULL|
|        489|           John Hill|travis59@sandoval...|            NULL|
|        577|       Tiffany Carey|gonzalezcody@will...|      2022-12-10|
+-----------+--------------------+-----------------

In [67]:
purchases.filter(purchases.customer_id == 577).show(100, truncate = False)

+-----------+-----------+---------------+-------------+----------------+--------------+
|purchase_id|customer_id|purchase_amount|purchase_date|product_category|payment_method|
+-----------+-----------+---------------+-------------+----------------+--------------+
|741        |577        |76.34          |2022-12-10   |Home & Kitchen  |Credit Card   |
+-----------+-----------+---------------+-------------+----------------+--------------+



In [70]:
sales = sparkSession.read.csv('/content/drive/MyDrive/Colab Notebooks/data/sales_data.csv',
                                  header = True,
                                  inferSchema = True)
sales.createOrReplaceTempView('sales')
sales.printSchema()
sales.show(5)

root
 |-- transaction_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- quantity_sold: integer (nullable = true)
 |-- price_per_unit: double (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- region: string (nullable = true)
 |-- payment_method: string (nullable = true)

+--------------+----------+---------------+-------------+--------------+----------------+-------------+--------------+
|transaction_id|product_id|   product_name|quantity_sold|price_per_unit|transaction_date|       region|payment_method|
+--------------+----------+---------------+-------------+--------------+----------------+-------------+--------------+
|             1|      1002|         Laptop|            1|        754.47|      2024-02-15|       Europe|    Debit Card|
|             2|      1003|     Headphones|            2|        692.86|      2023-01-01|       Africa|   Credit Card|
|             3|      1010|Fitness Tracker|

In [71]:
sqlTotalRevenue = '''SELECT SUM(quantity_sold * price_per_unit) as TotalRevenue
                     FROM sales'''
sparkSession.sql(sqlTotalRevenue).show()

+------------------+
|      TotalRevenue|
+------------------+
|2886615.8699999982|
+------------------+



In [72]:
sqlTop5SellingProducts = '''SELECT product_name, SUM(quantity_sold) as TotalQuantitySold
                            FROM sales
                            GROUP BY product_name
                            ORDER BY TotalQuantitySold DESC
                            LIMIT 5'''
sparkSession.sql(sqlTop5SellingProducts).show()

+-----------------+-----------------+
|     product_name|TotalQuantitySold|
+-----------------+-----------------+
|       Headphones|              729|
|       Smartwatch|              650|
|   Gaming Console|              646|
|           Camera|              611|
|Bluetooth Speaker|              568|
+-----------------+-----------------+



In [74]:
sqlRevenueByRegion = '''SELECT region, SUM(quantity_sold * price_per_unit) as TotalRevenue
                       FROM sales
                       GROUP BY region
                       ORDER BY TotalRevenue DESC'''
sparkSession.sql(sqlRevenueByRegion).show()

+-------------+-----------------+
|       region|     TotalRevenue|
+-------------+-----------------+
|North America|612992.0200000001|
|       Africa|        612526.04|
|         Asia|582756.7499999995|
|       Europe|542810.5700000002|
|South America|535530.4899999999|
+-------------+-----------------+



In [85]:
sqlRevenueByMonth = '''SELECT date_format(transaction_date, "yyyy-MM") as YearMonth,
                                SUM(quantity_sold * price_per_unit) as TotalRevenue
                       FROM sales
                       GROUP BY YearMonth
                       ORDER BY YearMonth ASC'''
sparkSession.sql(sqlRevenueByMonth).show()

+---------+------------------+
|YearMonth|      TotalRevenue|
+---------+------------------+
|  2022-10| 68568.84999999999|
|  2022-11|         128946.73|
|  2022-12|         111895.84|
|  2023-01| 82350.47000000002|
|  2023-02|         117857.88|
|  2023-03|         130579.66|
|  2023-04|142318.37999999998|
|  2023-05|132260.43999999997|
|  2023-06|100764.25999999998|
|  2023-07|          79193.52|
|  2023-08|123068.06000000001|
|  2023-09|136960.55000000002|
|  2023-10| 97569.38000000002|
|  2023-11|123324.64000000003|
|  2023-12|         151322.35|
|  2024-01| 92639.99999999999|
|  2024-02|127024.93999999997|
|  2024-03|135940.77000000002|
|  2024-04|114910.29000000001|
|  2024-05|126780.42000000001|
+---------+------------------+
only showing top 20 rows



In [87]:
sqlMostPopularPaymentMethodByRegion = '''WITH RankedPayments AS (
                                              SELECT region,
                                                  payment_method,
                                                  COUNT(*) as PaymentCount,
                                                  ROW_NUMBER() OVER (PARTITION BY region ORDER BY COUNT(*) DESC) as rn
                                              FROM sales
                                              GROUP BY region, payment_method
                                          )--End With

                                          SELECT region, payment_method, PaymentCount
                                          FROM RankedPayments
                                          WHERE rn = 1
                                          ORDER BY region'''
sparkSession.sql(sqlMostPopularPaymentMethodByRegion).show()

+-------------+--------------+------------+
|       region|payment_method|PaymentCount|
+-------------+--------------+------------+
|       Africa|   Credit Card|          47|
|         Asia|     Gift Card|          55|
|       Europe|        PayPal|          45|
|North America|        PayPal|          52|
|South America|        PayPal|          46|
+-------------+--------------+------------+



In [88]:
sqlAverageRevenueByRegion = '''SELECT region, avg(quantity_sold * price_per_unit) as AverageRevenue
                              FROM sales
                              GROUP BY region
                              ORDER BY AverageRevenue DESC'''
sparkSession.sql(sqlAverageRevenueByRegion).show()

+-------------+------------------+
|       region|    AverageRevenue|
+-------------+------------------+
|       Africa| 3017.369655172414|
|North America|2961.3141062801938|
|         Asia|2884.9344059405917|
|       Europe|2783.6439487179496|
|South America|2774.7693782383412|
+-------------+------------------+



In [89]:
sqlMostRevenueProduct = '''SELECT product_name, sum(quantity_sold * price_per_unit) as TotalRevenue
                           FROM sales
                           GROUP BY product_name
                           ORDER BY TotalRevenue DESC
                           LIMIT 1'''
sparkSession.sql(sqlMostRevenueProduct).show()

+------------+-----------------+
|product_name|     TotalRevenue|
+------------+-----------------+
|  Headphones|382734.3199999999|
+------------+-----------------+

